In [233]:
import PySpin
import matplotlib.pyplot as plt
import cv2
import numpy as np

### Configurar Câmera
Primeiro se configura a câmera utilizando do PySpin, depois deste bloco o objeto em cam está configurado e com método para capturar imagens

In [2]:
system = PySpin.System.GetInstance()
camera_list = system.GetCameras()
num_cam = camera_list.GetSize()
num_cam
cam = camera_list.GetByIndex(0)
cam.Init()
cam.TLStream.StreamBufferHandlingMode.SetValue(
PySpin.StreamBufferHandlingMode_NewestOnly
)

if not cam.IsStreaming():
    cam.BeginAcquisition()

### Get Image
O bloco abaixo utiliza da câmera acima para capturar uma imagem, a imagem fica armazenada em image_array

In [227]:
grabTimeout: int = PySpin.EVENT_TIMEOUT_INFINITE
streamID: int = 0

pyspin_image = cam.GetNextImage(grabTimeout, streamID)
image_array = pyspin_image.GetNDArray()

NameError: name 'cam' is not defined

Para testes com as imagens na pasta "../images"

In [228]:
#image_array = cv2.imread("../images/img0.jpeg")

### Convert to HSV
Para realizar a segmentação utilizamos de um threshold na escala HSV, portanto precisamos converter a imagem para esta escala

In [292]:
img = cv2.cvtColor(image_array, cv2.COLOR_RGB2HSV)

### Crop
Parte da imagem (fora da mesa) pode ser cortada para facilitar a segmentação

In [293]:
img = img[170:1150,275:1100]

### Threshold
Realizamos dois threshold, um para separar o braço robótico e outro para separar a bola. Nos dois utilizamos da hue como critério. Primeiro separamos o braço e guardamos em img1, depois separamos a bola e guardamos em img2, para posteriormente juntar as imagens em img

In [294]:
min_hue = 0
min_sat = 0
min_val = 0

max_hue = 10
max_sat = 255
max_val = 255


img1 = cv2.inRange(img, (min_hue, min_sat, min_val), (max_hue, max_sat, max_val))

In [295]:
copy = np.ndarray((img1.shape[0], img1.shape[1], 3))
for i in range (img1.shape[0]):
    for j in range(img1.shape[1]):
        if img1[i][j] != 0:
            copy[i][j] = [255,255,255]
        else:
            copy[i][j] = [0,0,0]

img1 = copy

In [296]:
min_hue = 110
min_sat = 0
min_val = 0

max_hue = 120
max_sat = 255
max_val = 255


img2 = cv2.inRange(img, (min_hue, min_sat, min_val), (max_hue, max_sat, max_val))

In [297]:
copy = np.ndarray((img2.shape[0], img2.shape[1], 3))

for i in range (img2.shape[0]):
    for j in range(img2.shape[1]):
        if img2[i][j] != 0:
            copy[i][j] = [0,0,255]
        else:
            copy[i][j] = [0,0,0]

img2 = copy

In [298]:
img = img1 + img2

### Resize
No bloco abaixo ocorre do downscale da imagem, em que reduzimos a resolução para igualar a resolução esperada pelo modelo treinado. TODO checar qual é a resolução certa

In [299]:
scale = (16,16)
img = cv2.resize(img, scale)

### Visualize
O bloco abaixo cria uma janela para visualizar a imagem img

In [1]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image', img)
cv2.waitKey(0)

NameError: name 'cv2' is not defined